In [ ]:
import numpy as np
import pandas as pd
from FinMind.data import DataLoader

merged_all = pd.DataFrame()

api = DataLoader()
api.login_by_token(api_token='xxx')

stock_pool = ["2330","2317","2454","2412","2308","6505","2303","2382",
              "1303","1301","2002","2207","1216","1326","3045","5871",
              "2395","2912","3008","1101","3037","3231","3034","4904",
              "2301","2357","2609","2408","3443","2618","2327","1590",
              "4938","2379","2603","2345","1402","9910","8046","2615",
              "2610","1102","2356","2376","1605","2377","3481","2409",
              "1476","2105","2324","2371","2474","1504","9945","2344",
              "2360","3017","1229","4958","9941","2347","2027","3533",
              "2353","9904","3702","2352","9921","2201","3023","3653",
              "2049","2383"]

In [ ]:
for stock_id in stock_pool:
    print(stock_id)
    df = api.taiwan_stock_financial_statement(
        stock_id=stock_id,
        start_date='2013-01-01',
    )
    # 使用 pivot_table 進行重塑
    df_financial_pivot = df.pivot_table(index=['date', 'stock_id'], columns='type', values='value', aggfunc='first').reset_index()
    df_financial_pivot.columns.name = None
    
    
    df = api.taiwan_stock_balance_sheet(
    stock_id=stock_id,
    start_date='2013-01-01',
    )
    df_balance_pivot = df.pivot_table(index=['date', 'stock_id'], columns='type', values='value', aggfunc='first').reset_index()

    df_balance_pivot.columns.name = None
    
    df = api.taiwan_stock_cash_flows_statement(
    stock_id=stock_id,
    start_date='2013-01-01',
    )
    df_cash_pivot = df.pivot_table(index=['date', 'stock_id'], columns='type', values='value', aggfunc='first').reset_index()

    df_cash_pivot.columns.name = None
    
    merged_df = df_financial_pivot.merge(df_balance_pivot, on=['date', 'stock_id']).merge(df_cash_pivot, on=['date', 'stock_id'])
    
    merged_all = pd.concat([merged_all, merged_df], ignore_index=True)

In [ ]:
merged_all

In [ ]:
merged_all = merged_all.dropna(axis=1)

In [ ]:
# 1. 加入.TW
# 2. 改日期 
#    3/31 改 5/16
#    6/30 改 8/15
#    9/30 改 11/15
#    12/31 改 4/1

In [ ]:
merged_all['stock_id'] = merged_all['stock_id'].apply(lambda x: x + '.TW')

In [ ]:
# 創建日期替换規則
date_rules = {
    '03-31': '05-16',
    '06-30': '08-15',
    '09-30': '11-15',
}

for rule in date_rules:
    old_date = rule
    new_date = date_rules[rule]
    merged_all['date'] = merged_all['date'].str.replace(old_date, new_date)
    


for year in range(2013, 2025):
    old_date = f'{year}-12-31'
    new_date = f'{year + 1}-04-01'
    merged_all['date'] = merged_all['date'].str.replace(old_date, new_date)

merged_all

In [ ]:
merged_all.to_csv('Financial statements.csv', index=False)

In [ ]:
import pandas as pd
import yfinance as yf
# 設定 CSV 檔案路徑
csv_file_path = 'Financial statements.csv'

Financial_df = pd.DataFrame()

# 讀取 CSV 檔案並轉換成 DataFrame
df = pd.read_csv(csv_file_path)

def append_tw_suffix(stock_id):
    if not stock_id.endswith('.TW'):
        stock_id += '.TW'
    return stock_id

# Use the apply method to modify the 'stock_id' column
df['stock_id'] = df['stock_id'].apply(append_tw_suffix)

# 顯示 DataFrame 資料
print(df)

In [ ]:
for stock_id in df['stock_id'].unique():

    #LSTM, RF, SVM, RL使用此資料(利用這一季到下一季的股價算Sharpe Ratio)
    selected_data = df[df['stock_id'] == stock_id].copy()
    selected_data['next_date'] = selected_data['date'].shift(-1)

    for index, row in selected_data.iterrows():
        if pd.isna(row['next_date']):
            continue;
        date = row['date'].replace("/", "-")
        next_date = row['next_date'].replace("/", "-")

        df_sin = yf.download(stock_id, start= date, end=next_date) 
        
        df_sin = df_sin[df_sin['Volume'] != 0]
        close = df_sin['Adj Close']
        pct_change = close.pct_change()
    
        profit = pct_change.mean()
        risk = pct_change.std()
        sharpe_ratio = profit/ risk
    
        selected_data.loc[index, 'sharpe_ratio'] = sharpe_ratio
        
    selected_data = selected_data.drop(['next_date'], axis=1, errors='ignore')        
    Financial_df = pd.concat([Financial_df, selected_data], ignore_index=True)

In [ ]:
Financial_df.to_csv('Stock-Picked Agent/Financial statements_SharpeRatio_RL.csv', index=False)

In [ ]:
for stock_id in df['stock_id'].unique():
    
    #GNN使用此資料(利用上一季到這一季的資料算Sharpe Ratio),為了要讓當季財報有夏普值可以訓練edge, 預測下一季夏普值無法訓練edge
    selected_data = df[df['stock_id'] == stock_id].copy()
    selected_data['before_date'] = selected_data['date'].shift(1)
    
    for index, row in selected_data.iterrows():
        if pd.isna(row['before_date']):
            continue;
        date = row['date'].replace("/", "-")
        before_date = row['before_date'].replace("/", "-")

        df_sin = yf.download(stock_id, start= before_date, end=date) 
        
        df_sin = df_sin[df_sin['Volume'] != 0]
        close = df_sin['Adj Close']
        pct_change = close.pct_change()
    
        profit = pct_change.mean()
        risk = pct_change.std()
        sharpe_ratio = profit/ risk
    
        selected_data.loc[index, 'sharpe_ratio'] = sharpe_ratio
        
    selected_data = selected_data.drop(['before_date'], axis=1, errors='ignore')
    
    stock_filename = stock_id[:4]
    file_path = f'GAT-main/data/{stock_filename}.csv'
    selected_data.to_csv(file_path, index=False)
        